<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Demo_RetentionTimeReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and imports




In [1]:
# install necessary packages
!python -m pip install -q dlomix==0.0.4
!python -m pip install -q wandb

# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.data import RetentionTimeDataset
from dlomix.models import PrositRetentionTimePredictor
from dlomix.models import RetentionTimePredictor
from dlomix.eval import TimeDeltaMetric

class Report():

  def __init__(self, project:str, title: str, description: str, dataset: RetentionTimeDataset):
    self.entity = wandb.apis.PublicApi().default_entity
    self.project = project
    self.title = title
    self.description = description
    self.api = wandb.Api()
    self.dataset = dataset
    self.table_key_len = ""
    self.table_key_rt = ""

  def create_report(self, add_config_section = True, add_data_section = True, add_train_section = True, add_val_section = True, add_train_val_section = True):
    report = wr.Report(
        project = self.project,
        title = self.title,
        description = self.description
    )

    report.blocks = [
        wr.TableOfContents()
    ]
    if add_config_section:
      report.blocks += self.config_section()
    if add_data_section:
      report.blocks += self.data_section()
    if add_train_section:
      report.blocks += self.train_section()
    if add_val_section:
      report.blocks += self.val_section()
    if add_train_val_section:
      report.blocks += self.train_val_section()

    report.save()



  # get metrics of last run in project or from specified run_id
  def get_metrics(self, run_id = None):
    if run_id:
      # run is specified by <entity>/<project>/<run_id>
      run = self.api.run(path = f"{self.entity}/{self.project}/{run_id}")
      metrics_dataframe = run.history()
      return metrics_dataframe
    else:
      # get metrics of latest run
      # api.runs seems to have a delay
      runs = self.api.runs(path = f"{self.entity}/{self.project}")
      run = runs[0]
      metrics_dataframe = run.history()
      return metrics_dataframe

  # get metric names split into train/val, train is further split into batch/epoch
  def get_metrics_names(self):
    metrics = self.get_metrics()
    # filter strings from list that are not starting with "_" and do not contain "val"
    pre_filter = [string for string in metrics if not string.startswith("_")]
    batch_train_metrics_names = [string for string in pre_filter if ("val" not in string.lower()) & ("epoch" not in string.lower()) & ("table" not in string.lower())]
    epoch_train_metrics_names = [string for string in pre_filter if ("val" not in string.lower()) & ("batch" not in string.lower()) & ("table" not in string.lower())]
    # filter strings from list that contain "val"
    epoch_val_metrics_names = list(filter(lambda x: "val" in x.lower(), metrics))
    # filter strings from train metrics that are 'epoch/learning_rate' and 'epoch/epoch'
    strings_to_filter = ['epoch/learning_rate', 'epoch/epoch', 'batch/learning_rate', 'batch/batch_step']
    batch_train_metrics_names = [string for string in batch_train_metrics_names if string not in strings_to_filter]
    epoch_train_metrics_names = [string for string in epoch_train_metrics_names if string not in strings_to_filter]
    batch_train_metrics_names.sort()
    epoch_train_metrics_names.sort()
    return batch_train_metrics_names, epoch_train_metrics_names, epoch_val_metrics_names

  def get_train_val_metrics_names(self):
    _, epoch_train_metrics_names, epoch_val_metrics_names = self.get_metrics_names()
    epoch_train_metrics_names.sort()
    epoch_val_metrics_names.sort()
    return list(zip(epoch_train_metrics_names, epoch_val_metrics_names))


  def config_section(self):
    config_block = [
        wr.H1(text = "Config"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels=[
            wr.RunComparer(layout = {'w': 24})
          ],
        ),
        wr.HorizontalRule(),
    ]
    return config_block

  def data_section(self):
    data_block = [
        wr.H1(text = "Data"),
        wr.P("The following section is showing a simple explorative data analysis of the used dataset. The first histogram shows the distribution of peptide lengths in the data set, while the second histogram shows the distribution of indexed retention times."),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels=[
            wr.CustomChart(
              query = {'summaryTable': {"tableKey" : self.table_key_len}},
              chart_name='master_praktikum/hist_pep_len',
              chart_fields={'value': self.dataset.sequence_col}
            ),
            wr.CustomChart(
              query = {'summaryTable': {"tableKey" : self.table_key_rt}},
              chart_name='master_praktikum/hist_ret_time',
              chart_fields={'value': self.dataset.target_col}
            )
          ]
        ),
        wr.HorizontalRule(),
    ]
    return data_block

  def train_section(self):
    batch_train_metrics_names, epoch_train_metrics_names, _ = self.get_metrics_names()
    panel_list_batch = []
    panel_list_epoch = []
    for name in batch_train_metrics_names:
      panel_list_batch.append(wr.LinePlot(x='Step', y=[name]))
    for name in epoch_train_metrics_names:
      panel_list_epoch.append(wr.LinePlot(x='Step', y=[name]))
    train_block = [
        wr.H1(text = "Training metrics"),
        wr.P("The following section shows the different metrics that were used to track the training. All used metrics are added by default. The first subsection shows the metrics per epoch, whereas the second subsection show the metrics per batch."),
        wr.H2(text = "per batch"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_batch
        ),
        wr.H2(text = "per epoch"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_epoch
        ),
        wr.HorizontalRule(),
    ]
    return train_block

  def val_section(self):
    _, _, epoch_val_metrics_names = self.get_metrics_names()
    panel_list_epoch = []
    for name in epoch_val_metrics_names:
      panel_list_epoch.append(wr.LinePlot(x='Step', y=[name]))
    val_block = [
        wr.H1(text = "Validation metrics"),
        wr.P("The following section shows the different metrics that were used to track the validation. All used metrics are added by default. The metrics are shown per epoch."),
        wr.H2(text = "per epoch"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_epoch
        ),
        wr.HorizontalRule(),
    ]
    return val_block

  def train_val_section(self):
    train_val_metrics_names = self.get_train_val_metrics_names()
    panel_list_epoch = []
    for name in train_val_metrics_names:
      panel_list_epoch.append(wr.LinePlot(x='Step', y=list(name)))
    train_val_block = [
        wr.H1(text = "Train - Validation metrics"),
        wr.P("The following section shows the training metrics in comparision with the validation metrics. All used metrics are added by default. The metrics are shown per epoch."),
        wr.H2(text = "per epoch"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_epoch
        ),
        wr.HorizontalRule(),
    ]
    return train_val_block

  # function to log sequence length table to wandb
  def log_sequence_length_table(self, data: pd.DataFrame, seq_col:str = "modified_sequence"):
    name_hist = "counts_hist"
    counts = self.count_seq_length(data, seq_col)
    # convert to df for easier handling
    counts_df = counts.to_frame()
    table = wandb.Table(dataframe = counts_df)
    # log to wandb
    hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_pep_len",
      data_table = table,
      fields = {"value" : seq_col}
    )
    wandb.log({name_hist: hist})
    name_hist_table = name_hist + "_table"
    return name_hist_table

  # function to count sequence length
  def count_seq_length(self, data: pd.DataFrame, seq_col: str) -> pd.Series:
      pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
      data[seq_col].replace(pattern, "", inplace= True)
      return data[seq_col].str.len()

  # function to log retention time table to wandb
  def log_rt_table(self, data: pd.DataFrame, rt_col:str = "indexed_retention_time"):
    name_hist = "rt_hist"
    rt = data.loc[:,rt_col]
    # convert to df for easier handling
    rt_df = rt.to_frame()
    table = wandb.Table(dataframe = rt_df)
    # log to wandb
    hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_ret_time",
      data_table = table,
      fields = {"value" : rt_col}
    )
    wandb.log({name_hist: hist})
    name_hist_table = name_hist + "_table"
    return name_hist_table

  def log_data(self):

    # check if datasource is a string
    if isinstance(self.dataset.data_source, str):
      # read corresponding file
      file_extension = self.dataset.data_source.split(".")[-1]
      match file_extension:
        case "csv":
          data = pd.read_csv(self.dataset.data_source)
        case "json":
          data = pd.read_json(self.dataset.data_source)
        case "parquet":
          data = pd.read_parquet(self.dataset.data_source, engine='fastparquet')
      self.table_key_len = self.log_sequence_length_table(data, self.dataset.sequence_col)
      self.table_key_rt = self.log_rt_table(data, self.dataset.target_col)

    # check if datasource is a tuple of two ndarrays or two lists
    if isinstance(self.dataset.data_source, tuple) and all(isinstance(item, (np.ndarray, list)) for item in self.dataset.data_source) and len(self.dataset.data_source) == 2:
      data = pd.DataFrame({self.dataset.sequence_col: self.dataset.data_source[0], self.dataset.target_col: self.dataset.data_source[1]})
      self.table_key_len = self.log_sequence_length_table(data, self.dataset.sequence_col)
      self.table_key_rt = self.log_rt_table(data, self.dataset.target_col)

    # check if datasource is a single ndarray or list
    # does not work? maybe error in RetentionTimeDataset
    if isinstance(self.dataset.data_source, (np.ndarray, list)):
      data = pd.DataFrame({self.dataset.sequence_col: self.dataset.data_source})
      self.table_key_len = self.log_sequence_length_table(data, self.dataset.sequence_col)


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


# Initialize Weights & Biases


In [9]:
# Create config
config = {
  "seq_length" : 30,
  "batch_size" : 64,
  "val_ratio" : 0.2,
  "lr" : 0.001,
  "optimizer" : "RMSprop",
  "loss" : "mse"
}

# Initialize WANDB
PROJECT = 'Demo_RetentionTimeReport'
RUN = "run_2"
wandb.init(project = PROJECT, name = RUN, config = config)

wandb: Currently logged in as: andi-obermeier (master_praktikum). Use `wandb login --relogin` to force relogin


# Load data

In [3]:
# load small train dataset
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'

# create dataset
rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = config["batch_size"],
                              val_ratio = config["val_ratio"],
                              test = False)

# Initialize report

In [4]:
# Create a report
report = Report(project = "Demo_RetentionTimeReport",
                title = "Comparison of different optimizers",
                description = "Comparison of two optimizers Adam and RMSprop",
                dataset = rtdata)

# Log data to Weights & Biases

In [5]:
report.log_data()

# Train model

In [10]:
# create retention time predictor
model = RetentionTimePredictor(seq_length = config["seq_length"])

# create the optimizer object
optimizer = tf.keras.optimizers.RMSprop(learning_rate = config["lr"])

# compile the model with the optimizer and the metrics we want to use
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

# train the model
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=15,
                    callbacks=[WandbMetricsLogger(log_freq = "batch")])

# finish wandb run
wandb.finish()

Epoch 1/15
424/424 [==============================] - 10s 19ms/step - loss: 571.6064 - mean_absolute_error: 17.5409 - timedelta: 18.1179 - val_loss: 195.7279 - val_mean_absolute_error: 10.4851 - val_timedelta: 9.5617
Epoch 2/15
424/424 [==============================] - 6s 15ms/step - loss: 255.1689 - mean_absolute_error: 12.1113 - timedelta: 13.4356 - val_loss: 153.8870 - val_mean_absolute_error: 9.0409 - val_timedelta: 8.6470
Epoch 3/15
424/424 [==============================] - 8s 19ms/step - loss: 213.0259 - mean_absolute_error: 11.0053 - timedelta: 11.7421 - val_loss: 129.5535 - val_mean_absolute_error: 8.1952 - val_timedelta: 8.2162
Epoch 4/15
424/424 [==============================] - 8s 18ms/step - loss: 184.5114 - mean_absolute_error: 10.1973 - timedelta: 10.6198 - val_loss: 113.7792 - val_mean_absolute_error: 7.5956 - val_timedelta: 7.6903
Epoch 5/15
424/424 [==============================] - 7s 16ms/step - loss: 161.5152 - mean_absolute_error: 9.4828 - timedelta: 10.2110 - v

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▃▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mean_absolute_error,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/timedelta,█▆▅▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁
epoch/mean_absolute_error,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁
epoch/timedelta,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
epoch/val_loss,█▆▄▃▃▂▂▂▅▁▂▁▁▁▁


# Create Report

In [7]:
report.create_report(add_data_section = True,
                     add_train_section = True,
                     add_val_section = True,
                     add_train_val_section = True)